In [1]:
import re
import collections
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import nltk


In [2]:
df = pd.read_csv('../datasets/raw/2020_06_23_CE-BVerwG_DE_Datensatz.csv')
df

,Unnamed: 0,doc_id,text,Gericht,Datum,Entscheidungsart,Senat,Verfahrensart,Ordinalzahl,Eingangsjahr,Suffix
0,1,BVerwG_1997-02-26_U_6_C_3_96_NA.txt,BUNDESVERWALTUNGSGERICHT\nIM NAMEN DES VOLKES\...,BVerwG,1997-02-26,U,6,C,3,96,NaN
1,2,BVerwG_2000-12-14_B_21_WB_107_00_NA.txt,BUNDESVERWALTUNGSGERICHT\nBeschluss\nBVerwG 1 ...,BVerwG,2000-12-14,B,21,WB,107,0,NaN
2,3,BVerwG_2002-01-10_B_9_A_9_02_NA.txt,BUNDESVERWALTUNGSGERICHT\nBESCHLUSS\nBVerwG 9 ...,BVerwG,2002-01-10,B,9,A,9,2,NaN
3,4,BVerwG_2002-01-14_B_4_BN_1_02_NA.txt,BUNDESVERWALTUNGSGERICHT\nBESCHLUSS\nBVerwG 4 ...,BVerwG,2002-01-14,B,4,BN,1,2,NaN
4,5,BVerwG_2002-01-17_B_1_B_12_02_NA.txt,BUNDESVERWALTUNGSGERICHT\nBESCHLUSS\nBVerwG 1 ...,BVerwG,2002-01-17,B,1,B,12,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...
24219,24220,BVerwG_2020-05-07_B_2_B_35_19_NA.txt,BESCHLUSS\nBVerwG 2 B 35.19\nOVG 4 B 33.14\n\n...,BVerwG,2020-05-07,B,2,B,35,19,NaN
24220,24221,BVerwG_2020-05-12_B_4_BN_3_20_NA.txt,BESCHLUSS\nBVerwG 4 BN 3.20\nOVG 2 C 285/18\n\...,BVerwG,2020-05-12,B,4,BN,3,20,NaN
24221,24222,BVerwG_2020-05-13_B_6_PKH_6_19_NA.txt,BESCHLUSS\nBVerwG 6 PKH 6.19\n\nIn der Verwalt...,BVerwG,2020-05-13,B,6,PKH,6,19,NaN
24222,24223,BVerwG_2020-05-18_B_1_B_21_20_NA.txt,"BESCHLUSS\nBVerwG 1 B 21.20, 1 PKH 12.20\nVGH ...",BVerwG,2020-05-18,B,1,B,21,20,NaN


In [124]:
from nltk.lm.preprocessing import padded_everygram_pipeline

def create_datasets(df, split=0.8):
    total_num_documents = 4
    data = []
    for judgement in df['text'][:total_num_documents]:
        data.extend(judgement.lower().split(" "))
    vocab = nltk.lm.Vocabulary(data, unk_cutoff=3)
    
    filtered_data = [word for word in data if word in vocab]
    print(len(filtered_data), len(data))

    # split to train and test
    split_index = round(len(filtered_data) * split)
    traindata = filtered_data[:split_index]
    testdata = filtered_data[split_index:]
    
    # create ngrams
    traindata = padded_everygram_pipeline(2, traindata)
    testdata = padded_everygram_pipeline(2, testdata)
    return list(traindata), testdata, vocab

In [133]:
traindata, testdata, vocab = create_datasets(df)


4176 6609


In [126]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(model, num_words, random_seed=42):
    """
    :param model: An ngram language model from `nltk.lm.model`.
    :param num_words: Max no. of words to generate.
    :param random_seed: Seed value for random.
    """
    content = []
    for token in model.generate(num_words, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)


In [127]:
counter = 0
for sample in testdata:
    print(sample)
    counter += 1
    words = generate_sent(model, 2, random_seed=7)
    # words = model.generate(3, text_seed=sample)
    print("prediction: ", [word for word in words])
    if counter > 10:
        break
   

<generator object padded_everygram_pipeline.<locals>.<genexpr> at 0x7fbfdfc10d60>


ValueError: Can't choose from empty population

In [135]:
n = 2
model = nltk.lm.MLE(n)
model.fit([traindata], vocabulary_text=vocab)
# model.fit([nltk.pad_sequence(traindata, 2, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>')])
print(len(model.vocab))
# model.generate(2, text_seed="b")

IndexError: tuple index out of range